In [5]:
import pandas as pd
from basic_api_access import BasicApiAccess
import datetime
import os
import numpy as np

In [6]:
username = os.environ['PEGELALARM_USER']
password = os.environ['PEGELALARM_PASSWORD']

baa = BasicApiAccess(username, password)

In [7]:
stations_dict = [{
    'water_name': 'Crana',
    'station_name': 'Tullyarvan'
}, {
    'water_name': 'Vichtbach',
    'station_name': 'Mulartshütte'
}, {
    'water_name': 'Losse',
    'station_name': 'Helsa'
}, {
    'water_name': 'Murg',
    'station_name': 'Frauenfeld'
}, {
    'water_name': 'Wupper',
    'station_name': 'Glüder'
}, {
    'water_name': 'Angerbach',
    'station_name': 'Ratingen'
}]
stations_df = pd.DataFrame.from_dict(stations_dict)
stations_df['common_id'] = None
stations_df


,water_name,station_name,common_id
0,Crana,Tullyarvan,None
1,Vichtbach,Mulartshütte,None
2,Losse,Helsa,None
3,Murg,Frauenfeld,None
4,Wupper,Glüder,None
5,Angerbach,Ratingen,None


In [8]:
for idx, row in stations_df.iterrows():
    currentStationData = baa.query_current_data(station_name=row['station_name'], water_name=row['water_name'])
    stations_df.at[idx, 'common_id'] = currentStationData["stations"][0]["commonid"]

stations_df.to_csv('./data/stations.csv', index=False)
stations_df

,water_name,station_name,common_id
0,Crana,Tullyarvan,39003-ie
1,Vichtbach,Mulartshütte,2824450000100-de
2,Losse,Helsa,42960105-de
3,Murg,Frauenfeld,2386-ch
4,Wupper,Glüder,2736731000100-de
5,Angerbach,Ratingen,2756500000100-de


In [9]:
stations_df = pd.read_csv('./data/stations.csv')
stations_df

,water_name,station_name,common_id
0,Crana,Tullyarvan,39003-ie
1,Vichtbach,Mulartshütte,2824450000100-de
2,Losse,Helsa,42960105-de
3,Murg,Frauenfeld,2386-ch
4,Wupper,Glüder,2736731000100-de
5,Angerbach,Ratingen,2756500000100-de


In [11]:
# Example to load historic water level height for a specific station
loadStartDate = datetime.datetime(2015, 1, 1, 00, 00)
loadEndDate = datetime.datetime.now()
ts_df_dict = {}
for idx, row in stations_df.iterrows():
    print(f'Requesting: {row["common_id"]}')
    df_raw = baa.query_historic_data(row['common_id'], loadStartDate, loadEndDate, granularity='raw')
    tmpEndDate = df_raw['sourceDate'].min()  - np.timedelta64(1, 's')
    df_hourly = baa.query_historic_data(row['common_id'], loadStartDate, tmpEndDate, granularity='hour')
    ts_df_dict[row['common_id']] = pd.concat([df_raw, df_hourly])

Requesting: 39003-ie
Requesting: 2824450000100-de
Requesting: 42960105-de
Requesting: 2386-ch
Requesting: 2736731000100-de
Requesting: 2756500000100-de


In [12]:
for key, val in ts_df_dict.items():
    print(f'Saving {key}')
    val.to_csv(f'./data/{key}.csv', index=False)

Saving 39003-ie
Saving 2824450000100-de
Saving 42960105-de
Saving 2386-ch
Saving 2736731000100-de
Saving 2756500000100-de
